# Import pre-annotations 

When you have a data pipeline with models outputting predictions, you want to be able to import those predicted annotations, or pre-annotations, into Label Studio for review and correction. 

In this example, use the [Label Studio SDK](https://labelstud.io/sdk/index.html) to write a Python script that transforms and imports predictions into a Label Studio project. 

## Connect to Label Studio

Connect to the Label Studio API using the Client module of the SDK:

In [3]:
from label_studio_sdk import Client

LABEL_STUDIO_URL = 'http://localhost:8081'
API_KEY = '67cd883ccb099c9d476e7d7601cd9a6e5fdfe323'

ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

## Create a project

Create a project for your labels. In this example, create an [image classification](https://labelstud.io/templates/image_classification.html) project:

In [4]:
project = ls.start_project(
    title='Project Created from SDK: Image Preannotation',
    label_config='''
    <View>
    <Image name="image" value="$image"/>
    <Choices name="image_class" toName="image">
        <Choice value="Cat"/>
        <Choice value="Dog"/>
    </Choices>
    </View>
    '''
)

## Import tasks with pre-annotations

You can import tasks with pre-annotations in several different ways. Choose one of these three methods for your script, based on how your model predictions are formatted.

### 1. Import tasks in Label Studio JSON format

You can format tasks in basic [JSON Label Studio format](https://labelstud.io/guide/tasks.html#Basic-Label-Studio-JSON-format) and choose to import the pre-annotations that way. 

In [5]:
project.import_tasks(
    [{
        'data': {'image': 'https://data.heartex.net/open-images/train_0/mini/0045dd96bf73936c.jpg'},
        'predictions': [{
            'result': [{
                'from_name': 'image_class',
                'to_name': 'image',
                'type': 'choices',
                'value': {
                    'choices': ['Dog']
                }
            }],
            'score': 0.87
        }]
    }, {
        'data': {'image': 'https://data.heartex.net/open-images/train_0/mini/0083d02f6ad18b38.jpg'},
        'predictions': [{
            'result': [{
                'from_name': 'image_class',
                'to_name': 'image',
                'type': 'choices',
                'value': {
                    'choices': ['Cat']
                }
            }],
            'score': 0.65
        }]
    }]
)

[36, 37]

> If you're not importing predictions for an image classification task, see the documentation for [importing pre-annotations](https://labelstud.io/guide/predictions.html) in Label Studio JSON format for more examples.

### 2. Import simple JSON predictions 

This simpler JSON format is a way to import pre-annotation results from a specific field for a single image. In this case, import task data with predictions in the `pet` field, and specify that the `pet` field contains the predicted classification:

In [6]:
project.import_tasks(
    [{'image': f'https://data.heartex.net/open-images/train_0/mini/0045dd96bf73936c.jpg', 'pet': 'Dog'},
    {'image': f'https://data.heartex.net/open-images/train_0/mini/0083d02f6ad18b38.jpg', 'pet': 'Cat'}],
    preannotated_from_fields=['pet']
)

[38, 39]

### 3. Import predictions from CSV

If your predictions are stored in CSV files, you can use pandas to read the dataframes:

In [7]:
!pip install pandas

In [8]:
import pandas as pd
pd.read_csv('data/images.csv')

,image,pet
0,https://data.heartex.net/open-images/train_0/m...,Dog
1,https://data.heartex.net/open-images/train_0/m...,Cat


Then you can specify the CSV file with the image references, and specify that the `pet` field contains the predicted classification:

In [9]:
project.import_tasks('data/images.csv', preannotated_from_fields=['pet'])

[40, 41]

### 4. Import predictions to existing tasks

In some cases, you may want to apply predictions to already imported tasks. For example, you can retrieve task from Label Studio, then create a new prediction:

In [10]:
tasks_ids = project.get_tasks_ids()
project.create_prediction(tasks_ids[0], result='Dog', model_version='1')

{'id': 52,
 'model_version': '1',
 'created_ago': '0\xa0minutes',
 'result': [{'from_name': 'image_class',
   'to_name': 'image',
   'type': 'choices',
   'value': {'choices': ['Dog']}}],
 'score': 0.0,
 'cluster': None,
 'neighbors': None,
 'mislabeling': 0.0,
 'created_at': '2021-11-15T20:44:06.902902Z',
 'updated_at': '2021-11-15T20:44:06.902902Z',
 'task': 36}

or bunch of predictions:

In [11]:
predictions = [{
    "task": tasks_ids[0],
    "result": "Dog",
    "score": 0.9
}, {
    "task": tasks_ids[1],
    "result": "Cat",
    "score": 0.8
}]
project.create_predictions(predictions)

{'created': 2}

In more complex annotation scenarios, check out [JSON format for expected predictions / preannotations](https://labelstud.io/guide/predictions.html)

## Annotate tasks in Label Studio

After importing pre-annotations using your preferred method, you can open Label Studio at http://localhost:8080 and correct and review the predictions and finish annotating your tasks.

## Calculate prediction accuracy

If you're using Label Studio Enterprise, you can calculate the accuracy of your predictions compared to the corrected annotations created as a ground truth dataset. 

Install and import the [evalme package](https://github.com/heartexlabs/label-studio-evalme) and calculate an agreement score for each task, comparing the annotation to the prediction:

In [12]:
!pip install label-studio-evalme

In [19]:
from evalme.metrics import get_agreement


print('Pre-annotation agreement scores:')

total_score = 0
n = 0
for task in project.get_project_tasks():
    score = get_agreement(task['annotations'][0], task['predictions'][0])
    print(f'{task["id"]} ==> {score}')
    total_score += score
    n += 1

print(f'Pre-annotation accuracy: {100 * total_score / n: .0f}%')

Pre-annotation agreement scores:
40 ==> 1.0
41 ==> 0.0
38 ==> 1.0
39 ==> 1.0
37 ==> 1.0
36 ==> 1.0
Pre-annotation accuracy:  83%


## Conclusion

With the Label Studio SDK, you can more easily import pre-annotated tasks into Label Studio so that you can create ground truth datasets, visually review the accuracy of predictions, and more. 

The `preannotated_from_fields` option for the `import_tasks()` method makes it easier to add your predictions without worrying about the intricacies of the Label Studio JSON format, but you can still use that field to add valuable metadata such as prediction scores and model versions to your pre-annotated task data. 